In [1]:
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps
from tqdm import tqdm

In [2]:
print("Conectando à base de dados")

Conectando à base de dados


In [3]:
MONGO_HOST = 'mongodb://localhost:27017/'

client = MongoClient(MONGO_HOST)

db = client['IWD2021']

tweets = db.tweets_server_updated

In [4]:
print(f'Base de dados: {db.name}\nCollection: {tweets.name}')

Base de dados: IWD2021
Collection: tweets_server_updated


In [5]:
pipeline = [
    {
        '$group': {
            '_id': '$id',
            'uniqueIds': {
                '$addToSet': '$_id'
            },
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$match': {
            'count': {
                '$gt': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]

In [6]:
print('Query iniciada')
data = list(tweets.aggregate(pipeline, allowDiskUse=True))
print('Query finalizada')

Query iniciada
Query finalizada


In [7]:
print(f'Quantidade de documentos com registros duplicados: {len(data)}')

Quantidade de documentos com registros duplicados: 3679164


In [8]:
print(dumps(data[0:2], indent=4, sort_keys=True, ensure_ascii=False))

[
    {
        "_id": 1368926517510737921,
        "count": 3,
        "uniqueIds": [
            {
                "$oid": "607cd2eae36d53cb43f2bd1e"
            },
            {
                "$oid": "605b500bc05c9bb8e47d5673"
            },
            {
                "$oid": "607cd2fbe36d53cb43f2bfe2"
            }
        ]
    },
    {
        "_id": 1368956153934315520,
        "count": 3,
        "uniqueIds": [
            {
                "$oid": "607cf01ae36d53cb43f76565"
            },
            {
                "$oid": "605b6f3ac05c9bb8e4824125"
            },
            {
                "$oid": "607cefffe36d53cb43f7613c"
            }
        ]
    }
]


In [9]:
ids = [d2 for d in data for d2 in d['uniqueIds'][1:]]

In [10]:
print(f'Quantidade de documentos a serem excluídos: {len(ids)}')

Quantidade de documentos a serem excluídos: 3679264


In [11]:
# ids

In [12]:
for id in tqdm(ids):
    tweets.delete_one({'_id': id})
    # print(f'Documento {id} removido')

print("Exclusão de documentos finalizadas.")

100%|██████████| 3679264/3679264 [16:40<00:00, 3676.21it/s]

Exclusão de documentos finalizadas.
